# Generating Primary Keys for Spanner

In Spanner databases, using monotonically increasing or decreasing primary keys is a potential performance problem. This is especially true for large databases with a lot of inserts. 
 
The best primary key is a natural key. This is where you choose two or more fields in the database that guarantee uniqueness. Just make sure the natural key doesn't always get larger (or smaller).
 
Sometimes a surrogate key is required. Traditionally, developers have used counters as surrogate primary keys. This is inefficient in Spanner. Spanner also has no built-in key generators, so it's up to the client, not the server, to generate the key.
 
There are several ways to generate surrogate keys. This notebook explore them. 


## Universally Unique Identifiers (UUIDs)

UUIDs are 36-bit strings guaranteed to be unique. There are different versions of UUIDs, so you must be careful. Version 1 UUIDs are generated with a timestamp. These always get larger, thus, should not be used for Spanner keys.
 
Version 4 UUIDs are generated using random numbers. These do not increase monotonically. The code below uses Python to generate 10 UUID strings. Notice the values do not continually get bigger. They are randomly distributed across the table space in a Spanner database, which is what you want. 


In [ ]:
import uuid

for i in range(10):
  print(str(uuid.uuid4()))

## Reversing the bits of integer values

Sometimes you are migrating a database that uses counters as keys. If so, you would need to preserve the existing primary-foreign key relations in the tables. One way to do that would be to reverse the bits in all the existing key values. 

The code below shows an algorithm for doing that. 

In [ ]:
def reverseBits(num,bitSize):
    # Convert number into binary representation
    # output will be like bin(10) = '0b10101'
    binary = bin(num)
    
    # Skip first two characters of binary
    # representation string and reverse
    # remaining string and then append zeros
    # after it. binary[-1:1:-1]  --> start
    # from last character and reverse it until
    # second last character from left
    reverse = binary[-1:1:-1]
    reverse = reverse + (bitSize - len(reverse))*'0'

    # converts reversed binary string into integer
    return  (int(reverse,2))  


values = [1, 2, 3, 4, 5, 50, 75, 100, 999, 10000]

print("Output:")
for value in values: 
  print(reverseBits(value,32))

## Convert the Binary Value to ASCII

Similar to the code above (and a bit easier to understand), you can use Python's built in binascii module to encode the counters in an existing database. 

In [ ]:
import binascii

values = [1, 2, 3, 4, 5, 50, 75, 100, 999, 10000]

print("Output:")
for value in values: 
  encoded = binascii.crc32(str(value).encode('utf8'))
  print(encoded)

## Hashing

Hashing takes any data and returns a hashed value. Because there are so many potential hashed values, it is extraordinarily unlikely that two different inputs would result in the same hashed value. Hashing works with any data type. It is a good way to use timestamps in your primary keys. Be aware that you can't reverse the operation. So, store the timestamps in another field if you want to be able to retrieve them later. 

In [ ]:
from datetime import datetime

for i in range(10):
  dt = datetime.now()         # Get the Date and time
  ts = datetime.timestamp(dt) # Get the Timestamp
  hashed = hash(str(ts))      # Hash the Timestamp
  print("Hashed Timestamp is:", hashed)
